In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
import scipy as sp
%matplotlib ipympl

plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})

In [2]:
GHz = 1e9
fF = 1e-15
nH = 1e-9

In [3]:
def fluxonium(LF, CF, EJ, nmax):
    loop = sq.Loop(0.5)
    C_01   = sq.Capacitor(CF, 'fF')
    L_01   = sq.Inductor (LF, 'nH', loops=[loop])
    EJ_01  = sq.Junction(EJ, 'GHz', loops=[loop])
    elements = {(0, 1): [L_01, C_01, EJ_01]}
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax])
    return circuit

def resonator(LR, CR, nmax):
    L_01   = sq.Inductor (LR, 'nH')
    C_01   = sq.Capacitor(CR, 'fF')
    elements = {(0, 1): [L_01, C_01],}
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax])
    return circuit

def coupled_unit_cells(LF1, CF1, EJ1, LR1, CR1, LF2, CF2, EJ2, LR2, CR2, CC, nmax):
    # This ignores the internal flux*flux coupling, that is for Δ=0. 
    loop1 = sq.Loop(0.5)
    C_01   = sq.Capacitor(CF1, 'fF')
    L_01   = sq.Inductor (LF1, 'nH', loops=[loop1])
    EJ_01  = sq.Junction (EJ1, 'GHz',loops=[loop1])

    L_02   = sq.Inductor (LR1, 'nH')
    C_02   = sq.Capacitor(CR1, 'fF')
    
    loop2 = sq.Loop(0.5)
    C_03   = sq.Capacitor(CF2, 'fF')
    L_03   = sq.Inductor (LF2, 'nH', loops=[loop2])
    EJ_03  = sq.Junction (EJ2, 'GHz',loops=[loop2])

    L_04   = sq.Inductor (LR2, 'nH')
    C_04   = sq.Capacitor(CR2, 'fF')
    
    C_C = sq.Capacitor(CC/4, 'fF') 
    
    elements = {(0, 1): [L_01, C_01, EJ_01],
                (0, 2): [L_02, C_02],
                (0, 3): [L_03, C_03, EJ_03],
                (0, 4): [L_04, C_04],
                (1, 3): [C_C],
                (1, 4): [C_C],
                (2, 3): [C_C],
                (2, 4): [C_C]}
    
    circuit = sq.Circuit(elements)
    circuit.set_trunc_nums([nmax,nmax,nmax,nmax])
    
    return circuit

In [4]:
Δ1  = 0
LF1 = 26.5
CF1 = 26.9
EJ1 = 5.4
ω_r1= 6.46
LR1 = 140
CR1 = 1/ (LR1* nH)  /(ω_r1*2*np.pi*GHz)**2/fF
Lq1, Lr1 = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF1, LR=LR1, Δ=Δ1)
C1, Csh1, CJ1 = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF1, CR=CR1)
φ_ext1=0.5

Δ2 = 0
LF2 = 20.4
CF2 = 22.4
EJ2 = 9.5
ω_r2 = 6.274
LR2 = 100
CR2 = 1/ (LR2* nH)  /(ω_r2*2*np.pi*GHz)**2/fF
Lq2, Lr2 = sq_ext.LF_LR_eff_to_Lq_Lr(LF=LF2, LR=LR2, Δ=Δ2)
C2, Csh2, CJ2 = sq_ext.CF_CR_eff_to_C_CJ_Csh(CF=CF2, CR=CR2)
φ_ext2 = 0.5

In [12]:
n_eig = 5
nmax = 8
CC = 10

only_inner = True
compensate_extra_cap = True

In [38]:
C0_mat = np.array([[CF1, 0, 0, 0],
                   [0, CR1, 0, 0],
                   [0, 0, CF2, 0],
                   [0, 0, 0, CR2]])

CC_mat = np.array([[CC / 2 , 0      , -CC / 4  , -CC / 4],
                   [0      , CC / 2 , -CC / 4  , -CC / 4 ],
                   [-CC / 4, -CC / 4, CC / 2   , 0        ],
                   [-CC / 4, -CC / 4, 0        , CC / 2  ]])
C_mat = C0_mat + CC_mat
C_inv = np.linalg.inv(C_mat)
# C_mat = sq_ext.C_mat_qubit_C_qubit(CC, CR=CR1, CF=CF1, CR_prime=CR2, CF_prime=CF2, only_inner=only_inner, compensate_extra_cap=compensate_extra_cap)
CF1_tilde = C_inv[0,0] ** -1
CR1_tilde = C_inv[1,1] ** -1
CF2_tilde = C_inv[2,2] ** -1
CR2_tilde = C_inv[3,3] ** -1
F1 = fluxonium(LF1, CF1_tilde, EJ1, nmax)
R1 = resonator(LR1, CR1_tilde, nmax)
F2 = fluxonium(LF2, CF2_tilde, EJ2, nmax)
R2 = resonator(LR2, CR2_tilde, nmax)

H_F1 = F1.hamiltonian()
H_R1 = R1.hamiltonian()
H_F2 = F2.hamiltonian()
H_R2 = R2.hamiltonian()

I_FR = qt.identity(H_F1.dims[0])
E_F1= sq_ext.diag(H_F1, n_eig, remove_ground=True)[0]
E_R1= sq_ext.diag(H_R1, n_eig, remove_ground=True)[0]
E_F2= sq_ext.diag(H_F2, n_eig, remove_ground=True)[0]
E_R2= sq_ext.diag(H_R2, n_eig, remove_ground=True)[0]

H_uc_1 =  qt.tensor(H_F1,I_FR) + qt.tensor(I_FR,H_R1)
H_uc_2 =  qt.tensor(H_F2,I_FR) + qt.tensor(I_FR,H_R2)

I_uc = qt.identity(H_uc_1.dims[0]) 
H_uncoupled_unit_cells_comp = qt.tensor(H_uc_1,I_uc) + qt.tensor(I_uc,H_uc_2)

Q_F1 = F1.charge_op(0)
Q_R1 = R1.charge_op(0)
Q_F2 = F2.charge_op(0)
Q_R2 = R2.charge_op(0)
Q_vec = [Q_F1, Q_R1, Q_F2, Q_R2]
H_coupling = 0

for i in range(4):
    for j in range(4):
        op_list = [I_FR, I_FR, I_FR, I_FR]
        if i == j: # we ommit the diagonal terms since we have already included the reonarmalizations (LR and LF tilde) in H_0.
            continue
        else:
            op_list[i] = Q_vec[i]
            op_list[j] = Q_vec[j]
            H_coupling += 1 * 1/2 * C_inv[i,j] * fF**-1 * qt.tensor(op_list)
            
H_coupled_unit_cells_comp = H_uncoupled_unit_cells_comp + H_coupling
E_coupled_unit_cells_comp = sq_ext.diag(H_coupled_unit_cells_comp, n_eig, remove_ground=True)[0]
E_coupled_unit_cells_comp

In [281]:
H_coupled_unit_cells_full = coupled_unit_cells(LF1, CF1, EJ1, LR1, CR1, LF2, CF2, EJ2, LR2, CR2, CC, nmax).hamiltonian()
E_coupled_unit_cells_full = sq_ext.diag(H_coupled_unit_cells_full, n_eig, remove_ground=True)[0]
E_coupled_unit_cells_full

In [282]:
H_qubit_C_qubit_full = (sq_ext.sq_qubit_C_qubit(CC, C1, CJ1, Csh1, Lq1, Lr1, Δ1, EJ1, C2, CJ2, Csh2, Lq2, Lr2, Δ2, EJ2, 
                                               nmax_r=nmax, nmax_f=nmax, only_inner = only_inner, compensate_extra_cap = compensate_extra_cap)
                        .hamiltonian())
E_qubit_C_qubit_full = sq_ext.diag(H_qubit_C_qubit_full, n_eig, remove_ground=True)[0] 
E_qubit_C_qubit_full

In [283]:
H_qubit_C_qubit_comp = sq_ext.hamiltonian_qubit_C_qubit(CC, CR1, CF1, LF1, LR1, EJ1, Δ1, φ_ext1, 
                                                        CR2, CF2, LF2, LR2, EJ2, Δ2, φ_ext2,
                                                        nmax_r=nmax, nmax_f=nmax, only_inner = only_inner, compensate_extra_cap = compensate_extra_cap)
E_qubit_C_qubit_comp = sq_ext.diag(H_qubit_C_qubit_comp, n_eig, remove_ground=True)[0] 
E_qubit_C_qubit_comp

In [284]:
E_qubit_C_qubit_full, E_coupled_unit_cells_full, E_qubit_C_qubit_comp, E_coupled_unit_cells_comp